# Who Got Scott Walker's Donors?

The code below looks for donors who gave to Scott Walker (before he dropped out of the 2016 presidential race) and are now giving to other candidates. A few notes and caveats:

- The analysis focuses on __inidividual contributors__ to the __official committees__ of each campaign. It does not examine organizational contributors or contributions to Super PACs, leadership committees, or other candidate-support groups.


- The Federal Election Commission does not assign unique IDs to contributors. As an approximation, this code considers two contributions to have come from __the same donor if they share all three of the following attributes__:
    - Last name
    - First name
    - 5-digit ZIP code


- Because of the lack of unique IDs, and the possibility that donors change name-spellings or ZIP codes between contributions, this method may not capture all donor movement. Additionally, if two donors with the same first and last name live in the same ZIP code, they will be mistakenly labeled as the same person.


- A candidate is considered to have __picked up a Walker donor__ if that donor:
    - Gave to Walker's official committee before Sept. 21.
    - Did not give to the other candidate's committee since April 1 and before Sept. 21.
    - *Did* give to the other candidate's committee on Sept. 21 or later.

In [1]:
import pandas as pd
import io
import glob

## Define committee names and IDs

In [2]:
WALKER = "Scott Walker, Inc."
BUSH = "Jeb 2016, Inc."
RUBIO = "Marco Rubio for President"
CRUZ = "Cruz for President"
FIORINA = "Carly for President"

In [3]:
committees = {
    "C00580480": WALKER,
    "C00579458": BUSH,
    "C00458844": RUBIO,
    "C00574624": CRUZ,
    "C00577312": FIORINA
}

## Load contributions from two most recent quarterly filings

For the official committees supporting Walker, Bush, Rubio, Cruz, and Fiorina, we load the two most recent quarterly filings — the July and October filings. The one exception is Walker; his committee officially began in July, and so only has an October quarterly filing.

In [4]:
filings = glob.glob("../data/fec-filings/*.fec")

In [5]:
def profile_filing(path):
    with open(path) as f:
        header = f.readline()
        f3 = pd.read_csv(io.StringIO(f.readline()), header=None).iloc[0]
    return "|".join(map(str, [committees[f3[1]], f3[15], f3[16], path]))

In [6]:
for p in sorted(map(profile_filing, filings)):
    print(p)

Carly for President|20150401|20150630|../data/fec-filings/1015551.fec
Carly for President|20150701|20150930|../data/fec-filings/1029240.fec
Cruz for President|20150401|20150630|../data/fec-filings/1024046.fec
Cruz for President|20150701|20150930|../data/fec-filings/1029462.fec
Jeb 2016, Inc.|20150401|20150630|../data/fec-filings/1024106.fec
Jeb 2016, Inc.|20150701|20150930|../data/fec-filings/1029571.fec
Marco Rubio for President|20150401|20150630|../data/fec-filings/1029436.fec
Marco Rubio for President|20150701|20150930|../data/fec-filings/1029457.fec
Scott Walker, Inc.|20150617|20150930|../data/fec-filings/1028979.fec


In [7]:
with open("../data/contrib-fields.txt") as f:
    contrib_fields = f.read().strip().split("\n")

In [8]:
def get_indiv_contributions(path):
    with open(path) as f:
        sa17a = (line for line in f if line[:7] == '"SA17A"')
        contributions = pd.read_csv(io.StringIO("\n".join(sa17a)),
            header=None,
            names=contrib_fields,
            parse_dates=["contribution_date"],
            dtype={ "contributor_zip": str },
            low_memory=False)
    return contributions[
        contributions["entity_type"] == "IND"
    ].copy()

In [9]:
contributions = pd.concat([ get_indiv_contributions(f) for f in filings ])

In [10]:
contributions["committee"] = contributions["filer_committee_id_number"].apply(committees.get)

In [11]:
contributions["zip_5"] = contributions["contributor_zip"].fillna("").apply(lambda x: x[:5])

In [12]:
print("{0:,d} total contributions".format(len(contributions)))

118,760 total contributions


### Total \$\$\$ for Q2 and Q3 *itemitzed* individual contributions. 

This number does not reflect the total amount raised by the commmittees; it does not, for example, include donations from organizations; small, "unitemized" donations; or contributions from other committees.

In [13]:
contributions.groupby("committee")["contribution_amount"].sum().sort_values(ascending=False)

committee
Jeb 2016, Inc.               23226422.85
Cruz for President           14746552.76
Marco Rubio for President    11313202.08
Scott Walker, Inc.            4691851.54
Carly for President           4540582.06
Name: contribution_amount, dtype: float64

## Categorize donations as pre- or post- Walker dropout

Note: Contributions on the precise date of Walker's dropout are marked as *post*-dropout.

In [14]:
walker_dropout_date = "2015-09-21"

In [15]:
pre_dropout = (contributions["contribution_date"] < walker_dropout_date)
contributions["walker_phase"] = pre_dropout.apply(lambda x: "pre_dropout" if x else "post_dropout")

## Add donor "index" as a substitute for unique ID

In [16]:
def make_donor_index(donor):
    is_null = lambda x: pd.isnull(x) or (len(x) == 0)
    if is_null(donor["zip_5"]) \
    or is_null(donor["contributor_first_name"]) \
    or is_null(donor["contributor_last_name"]):
        return None
    fmt = "{contributor_last_name}|{contributor_first_name}|{zip_5}"
    return fmt.format(**donor).upper()

In [17]:
contributions["donor_index"] = contributions.apply(make_donor_index, axis=1)

In [18]:
donor_lists = contributions.dropna(subset=["donor_index"])\
    .groupby([ "committee", "walker_phase" ])\
    ["donor_index"].unique().apply(set)
donor_lists

committee                  walker_phase
Carly for President        post_dropout    set([WILLIAMS|BRENT|84121, BILDEN|PETER|06840,...
                           pre_dropout     set([MELLO|ED|75230, ANDREW|KEVIN|93309, KNAUP...
Cruz for President         post_dropout    set([PORTWINE|KENNETH|63119, SIEGEL|MATTHEW|66...
                           pre_dropout     set([PORTWINE|KENNETH|63119, BUTLER|CYNTHIA|01...
Jeb 2016, Inc.             post_dropout    set([STRAWN|PAIGE|98119, GREGG|DIANE|32305, MC...
                           pre_dropout     set([BIBER|KATHRYN|94122, BOYD|JEFF|06830, ARM...
Marco Rubio for President  post_dropout    set([GLASER|PATRICIA|90067, TEGAN|JAMES|55447,...
                           pre_dropout     set([MELLA|DIEGO|33186, MCLELLAN|JAMES|32533, ...
Scott Walker, Inc.         post_dropout    set([AMOS|RALPH|46001, KELLY|LOIS|48334, FELTE...
                           pre_dropout     set([PUTZ|CHRISTI|83703, GEARHART|MARILYN|9885...
Name: donor_index, dtype: obje

In [19]:
def get_walker_pickups(committee):
    new_donors = donor_lists[committee]["post_dropout"] - donor_lists[committee]["pre_dropout"]
    overlap = new_donors.intersection(donor_lists[WALKER]["pre_dropout"])
    pickups = contributions[
        contributions["donor_index"].isin(overlap) &
        (contributions["committee"] == committee) &
        (contributions["walker_phase"] == "post_dropout")
    ].groupby("donor_index")["contribution_amount"].sum()
    return pickups.sort_values(ascending=False)

In [20]:
def summarize_walker_pickups(committee):
    pickups = get_walker_pickups(committee)
    print((
        "{0} picked up approx. {1} Walker donors, "
        "who've contributed ${2:,.0f} since Sept. 21. They are:\n"
    ).format(committee, len(pickups), pickups.sum()))
    print(pickups)

In [21]:
summarize_walker_pickups(BUSH)

Jeb 2016, Inc. picked up approx. 17 Walker donors, who've contributed $40,750 since Sept. 21. They are:

donor_index
HUBBARD|STANLEY|55114       5400
GRABER|RICHARD|22314        5400
HUBBARD|KAREN|55043         4700
WEINMANN|JOHN|70130         2700
BROCK|DONNIE|74104          2700
BROCK|JOHN|74104            2700
MIZEL|LARRY|80237           2700
RUNDE|DANIEL|22101          2700
RUNDE|SONIA|22101           2700
SCARAMUCCI|ANTHONY|11030    2700
SCARAMUCCI|DEIDRE|11030     2700
SAMSON|MICHAEL|94121        1000
BIERMAN|BROCK|20141         1000
ROBERTS|JACK|99361           500
DAVIS|DONALD|94904           500
GREAVES|WILLIAM|53217        350
STOKES|LINDSEY|63132         300
Name: contribution_amount, dtype: float64


In [22]:
summarize_walker_pickups(RUBIO)

Marco Rubio for President picked up approx. 33 Walker donors, who've contributed $33,216 since Sept. 21. They are:

donor_index
OBERWEIS|JIM|60554               5380
BAILEY|MONA|76180                2700
LOCKE|AIMEE|78212                2700
COLLINS|DANIEL|91001             2700
CONKEY|MARY|95746                2700
HANNA|COLIN|19382                2700
JORDAN|THOMAS|95448              2700
HUBBARD|STANLEY|55114            2500
BARNARD|TIMOTHY|59771            1000
RICHARDSON|ROBERT|46516          1000
MUZZI|VINCENT|94030              1000
UIHLEIN|LUCIA|34228              1000
HALLAS|LAURENCE|63011             500
KORPAN|RICHARD|80439              500
KRESS|DONALD|54307                500
TEGAN|JAMES|55447                 500
ROBERTSON|MARY|06069              500
MCINVALE|GERALD|30240             250
MORTON|RICHARD|33418              250
TESTA|DAVID|34491                 250
DAVIS|LARRY|75228                 250
GIESZL|YALE|81615                 250
SHERRED|ELLEN|32459               25

In [23]:
summarize_walker_pickups(CRUZ)

Cruz for President picked up approx. 26 Walker donors, who've contributed $25,355 since Sept. 21. They are:

donor_index
WESTCOTT|CHART|75201       5400
WESTCOTT|CARL|75201        5400
ISAAC|PAUL|10538           2700
KADISHA|NEIL|90212         2000
WHITE|WALTER|77024         1700
NAEGELE|ELLIS|34108        1000
SAMELMAN|IRWIN|89113       1000
STIRRATT|AVERY|06840        500
HOMMES|GERALD|55446         500
RABY|LON|35802              500
PRATT|CHARLES|60093         500
HAAS|ALISON|07821           500
FLOOD|MARYANN|98683         500
KRAUSE|CHARLES|03458        500
FOGARTY|ROSALEE|94028       350
TOFT|KAREN|15235            300
GROSVENOR|DANNA|80123       250
PATTEN|DAVID|19083          250
SCHNEEBECK|ROBERT|34223     250
BLACK|CYNTHIA|93311         250
SULLIVAN|ALFRED|98040       250
DAAS|STEPHEN|55418          250
BRYDEN|ELIZABETH|10023      205
MASTERSON|CONRAD|75219      100
SMITH|WILLIAM|03811         100
BELCHER|WILLIAM|95008       100
Name: contribution_amount, dtype: float64


In [24]:
summarize_walker_pickups(FIORINA)

Carly for President picked up approx. 27 Walker donors, who've contributed $24,100 since Sept. 21. They are:

donor_index
BECKWITH|GEORGE|15238    2700
BUSCH|AUGUST|63376       2700
MASON|JIM|98520          2700
WITMER|TODD|17552        2500
HUBBARD|STANLEY|55114    2500
CROSS|DEVON|10065        1000
DOWD|JOHN|22182          1000
ELDREDGE|KEITH|17315     1000
MCBRIDE|JAMES|04268      1000
PORTER|RICHARD|60093      700
JENNINGS|VERN|89011       500
EARHART|ALAN|94024        500
GRUCHALA|PAUL|48185       500
HARRIS|BOBBYE|30701       500
HOUSTON|REAGAN|78279      500
KEEN|CLAY|88065           500
RATH|DIANE|78209          500
ROBERTS|JACK|99361        500
SAMSON|MICHAEL|94121      500
JONES|BRYAN|55330         300
MCMANUS|VINCENT|06492     300
SWENSON|ROBERT|53546      250
MCCUNE|JOHN|67226         250
MORTON|YOLANDA|91387      250
BARNHART|NANCY|38053      250
SCHAUS|NICHOLAS|32963     100
SCHOENECK|PAUL|53122      100
Name: contribution_amount, dtype: float64


---

---

---